In [1]:
pip install PyMuPDF


  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/4a/26/8c72973b8833a72785cedc3981eb59b8ac7075942718bbb7b69b352cdde4/pymupdf-1.26.3-cp39-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/18.7 MB 1.2 MB/s eta 0:00:16
   ---------------------------------------- 0.1/18.7 MB 1.2 MB/s eta 0:00:16
   ---------------------------------------- 0.2/18.7 MB 1.1 MB/s eta 0:00:18
    --------------------------------------- 0.2/18.7 MB 1.1 MB/s eta 0:00:17
    --------------------------------------- 0.3/18.7 MB 1.0 MB/s eta 0:00:19
    --------------------------------------- 0.3/18.7 MB 983.9 kB/s eta 0:00:19
    --------------------------------------- 0.3/18.7 MB 905.4 kB/s eta 0:00:21
    --------------------------------------- 0.3/18.7 MB 840.2 kB/s eta 0:00:22
    ---------------------------

In [2]:
import fitz  # PyMuPDF

input_pdf = "visitors.pdf"
output_pdf = "visitors_new.pdf"

doc = fitz.open(input_pdf)

# Reduce image quality for each page
for page in doc:
    pix = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)
    page.clean_contents()  # Clean previous high-res images
    page.insert_image(page.rect, pixmap=pix)

doc.save(output_pdf)
doc.close()

print(" Compressed PDF saved as visitors_compressed.pdf")


✅ Compressed PDF saved as visitors_compressed.pdf


In [3]:
pip install pdf2image


  Obtaining dependency information for pdf2image from https://files.pythonhosted.org/packages/62/33/61766ae033518957f877ab246f87ca30a85b778ebaad65b7f74fa7e52988/pdf2image-1.17.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pdf2image import convert_from_path

pdf_path = "visitors.pdf"
images = convert_from_path(pdf_path)

# Save only the first page to test
images[0].save("page1.jpg", "JPEG")
print(" First page saved as image.")


✅ First page saved as image.


In [2]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://visitors500description.cognitiveservices.azure.com/"
key = "ooo"

client = DocumentAnalysisClient(endpoint, AzureKeyCredential(key))

with open("page1.jpg", "rb") as f:
    poller = client.begin_analyze_document("prebuilt-document", document=f)
    result = poller.result()

print("Azure processed the image successfully!")


✅ Azure processed the image successfully!


In [3]:
import pandas as pd

# Create a list to hold all tables
all_tables = []

# Loop through each table in the result
for i, table in enumerate(result.tables):
    data = []

    # Extract each row of the table
    for row_idx in range(table.row_count):
        row_data = []
        for col_idx in range(table.column_count):
            # Find the cell at this row/column
            cell = next(
                (c for c in table.cells if c.row_index == row_idx and c.column_index == col_idx),
                None
            )
            row_data.append(cell.content if cell else "")
        data.append(row_data)

    # Convert the current table to DataFrame
    df = pd.DataFrame(data)
    all_tables.append((f"Table_{i+1}", df))

# Save to Excel file
with pd.ExcelWriter("output_tables.xlsx") as writer:
    for sheet_name, df in all_tables:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(" Tables extracted and saved to output_tables.xlsx!")


✅ Tables extracted and saved to output_tables.xlsx!


In [4]:
# Display the first table
all_tables[0][1]


,0,1,2,3
0,DAY-1 Date,"Name "" Address",E. Mail,Feedback
1,10.07.255,"Karthick. (Krishnagiri),",94802 71417.,15 acres Farming. (all crops).
2,10.07.25.\n2),Balasubramaniam (COCOTRAP)\nTANJORE. :selected...,9442111366.,COCONUT (BORK).
3,3,Rajía (Salem).,99 652 30 846.,All crops
4,4),Shankar. Truvanamalai,9943 591283,Trubblegum Roban.
5,5),Senthil (Arakonan),96 77 180 550-,Coco Trap.
6,6) 1,Rajque (Tanjore),90431 34089,coco trap
7,7),Vanjappon. (vallonfoie),9443925549,Coconut-
8,8),Gopi (Burani),99 767 53 469.,Coco Trop arguing


In [5]:
from pdf2image import convert_from_path
import os
import pandas as pd
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from PIL import Image
from io import BytesIO

# Azure credentials
endpoint = "https://visitors500description.cognitiveservices.azure.com/"
key = "ooooooooooo"
client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))

# Convert full PDF to images
pdf_path = "visitors.pdf"
images = convert_from_path(pdf_path)

# List to hold all tables
combined_data = []

# Process each page
for page_num, image in enumerate(images, start=1):
    # Convert image to bytes
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format="PNG")
    img_byte_arr.seek(0)

    # Analyze the image
    poller = client.begin_analyze_document("prebuilt-layout", document=img_byte_arr)
    result = poller.result()

    for i, table in enumerate(result.tables):
        data = []

        for row_idx in range(table.row_count):
            row_data = []
            for col_idx in range(table.column_count):
                cell = next(
                    (c for c in table.cells if c.row_index == row_idx and c.column_index == col_idx),
                    None
                )
                row_data.append(cell.content if cell else "")
            data.append(row_data)

        df = pd.DataFrame(data)
        df.insert(0, "Page", page_num)
        df.insert(1, "Table", i + 1)
        combined_data.append(df)

# Combine all into one DataFrame
final_df = pd.concat(combined_data, ignore_index=True)

# Save to a single Excel sheet
final_df.to_excel("Combined_Tables_From_PDF.xlsx", index=False)

print(" All tables combined and saved to Combined_Tables_From_PDF.xlsx")


✅ All tables combined and saved to Combined_Tables_From_PDF.xlsx


In [6]:
# Import the required library
import pandas as pd

# Load the saved Excel file
df_loaded = pd.read_excel("Combined_Tables_From_PDF.xlsx")

# Display the first few rows
df_loaded.head()  # You can also use df_loaded.tail() to see the bottom rows


,Page,Table,0,1,2,3
0,1,1,DAY-1 Date,"Name "" Address",E. Mail,Feedback
1,1,1,10.07.255,Karthick. (Krishnagiri).,94802 71417.,15 acres Farming. (all crops).
2,1,1,10.07.25. 2),Balasubramaniam (COCOTRAP) TANJORE.\n:selected:,9442111366.,COCONUT (BORK).
3,1,1,LE .,Raja (Salem).,99 652 30 846.,Allcrops .
4,1,1,4),Shankar. Truvanamalai,9943 591283,Trubblegum Roban.
